# Loading data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

df = pd.read_csv('Comparably_reviews.csv')
reviews = df['Review_text']
len(reviews)

3790

# Downloads
- NLTK for stopwords
- Scapy for lemmatization

In [3]:
%%shell

# Run in terminal or command prompt
python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Importing packages

In [5]:
! pip install pyLDAvis
! pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [6]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


# Prepare Stopwords

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
len(stop_words)

184

# Clean the texts

In [8]:
# Convert to list
data = df.Review_text.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:10])

['Overall good for learning, but pay is low',
 'Very good Onsite opportunities and perks at onsite',
 'Very good culture , I get good work exposure and get to work with good team',
 'Horrible. Unfit Project Managers Useless HR Unprofessional RMG',
 'Brilliant, Valuable, Learning, Sharing, Caring',
 'Encouraging, motivating, allows to take new challenge',
 'Onsite opportunities and career growth',
 'Team is encouraging and very supportive.',
 'Excellent Company to Work for',
 'Greetings! I love TCS Leadership Team for making us feel like a big family '
 ':)']


In [9]:
# Tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        # deacc=True removes punctuations

data_words = list(sent_to_words(data))

pprint(data_words[:2])

[['overall', 'good', 'for', 'learning', 'but', 'pay', 'is', 'low'],
 ['very', 'good', 'onsite', 'opportunities', 'and', 'perks', 'at', 'onsite']]


# Lemmatization

In [10]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['overall good learning pay low', 'very good onsite opportunity perk onsite']


# Document Word Matrix

In [11]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

Sparsicity is the percentage of non-zero datapoints in the document-word matrix

In [12]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsity:  0.9797846539271342 %


# Topic Model

In [14]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


# Evaluate
- higher log likelihood
- lower perplexity

In [15]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -91535.63782041507
Perplexity:  373.2829889507531
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


# Find Best LDA Model

In [23]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv=None, error_score=nan,
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                                 tota

In [24]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -21198.600620337675
Model Perplexity:  306.58848605917876


# Dominant topic in each review

In [26]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.200000,0.020000,0.020000,0.020000,0.670000,0.020000,0.020000,0.020000,0.020000,0.020000,4
Doc1,0.030000,0.030000,0.030000,0.770000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,3
Doc2,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.890000,0.010000,0.010000,7
Doc3,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,6
Doc4,0.700000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0
Doc5,0.020000,0.020000,0.820000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,2
Doc6,0.030000,0.030000,0.770000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,2
Doc7,0.030000,0.770000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,1
Doc8,0.030000,0.030000,0.770000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,2
Doc9,0.010000,0.010000,0.010000,0.010000,0.150000,0.310000,0.010000,0.280000,0.010000,0.180000,5


In [27]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,0,558
1,4,474
2,1,396
3,5,386
4,2,379
5,9,341
6,6,337
7,8,332
8,7,296
9,3,291


# Visualization

In [28]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
4      32.420952   8.115230       1        1  11.831714
5       0.648955  71.707100       2        1  11.469259
1      71.019684   9.971006       3        1  11.315988
0       3.785981  33.185135       4        1  11.302987
8     -42.666500  -9.222070       5        1  10.102929
2      -3.621689 -43.526516       6        1  10.087567
7      -3.632402  -4.171708       7        1   8.573822
6     -36.584400  35.804413       8        1   8.539540
9      44.391380  50.615383       9        1   8.485911
3      40.235809 -31.591665      10        1   8.290284, topic_info=         Term        Freq       Total Category  logprob  loglift
352      team  385.000000  385.000000  Default  30.0000  30.0000
254       pay  251.000000  251.000000  Default  29.0000  29.0000
236      need  311.000000  311.000000  Default  28.0000  28.0000
388      work  792.000000  792.000000  Default  27.0000  27.0000
255    people  378.000000  378.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
148     great   33.107679  187.547139  Topic10  -3.6559   0.7558
55    company   21.250537  259.423577  Topic10  -4.0993  -0.0120
98   employee   23.529310  397.959226  Topic10  -3.9974  -0.3380
23       base   12.576517   52.708476  Topic10  -4.6238   1.0571
36       care   13.959580  129.686197  Topic10  -4.5195   0.2611

[443 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.092414  ability
0         2  0.138620  ability
0         3  0.231034  ability
0         5  0.092414  ability
0         9  0.462068  ability
...     ...       ...      ...
395       1  0.209304     year
395       2  0.348840     year
395       3  0.104652     year
395       4  0.122094     year
395       5  0.191862     year

[867 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 2, 1, 9, 3, 8, 7, 10, 4])

# Topic Keywords

In [29]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,ability,able,action,activity,actually,advancement,allow,amazing,annual,answer,appreciate,appreciation,approach,area,ask,associate,atmosphere,attention,attitude,available,awesome,bad,balance,base,basic,basis,believe,benefit,better,big,bonus,boss,break,bring,build,business,care,career,ceo,certain,...,technology,tell,term,test,thing,think,time,total,train,training,transparency,transparent,treat,trust,try,understand,understanding,update,use,vacation,value,view,vision,wage,wait,walk,want,way,week,willing,willingness,woman,work,worker,working,workplace,world,worth,wrong,year
Topic0,0.100013,0.100003,0.100014,0.100002,10.148370,2.418462,0.100009,0.100002,0.100003,0.100015,2.470072,0.100000,5.100032,0.100003,0.100002,0.100011,0.100000,0.100010,0.100038,0.100000,0.100000,24.713132,0.100001,14.583110,3.878762,0.100000,0.100008,0.100004,1.569309,3.099933,0.100005,0.100000,0.100004,0.100015,0.100000,0.100013,117.739042,0.100007,0.100020,0.100003,...,0.100006,12.666979,21.901952,13.037780,5.430430,39.180079,0.100039,0.100000,0.100010,4.404163,35.079168,0.100006,0.124767,0.100008,0.100028,0.100014,20.625567,0.100001,4.866109,0.104440,0.100010,5.099978,6.153049,0.100001,0.100007,2.797292,22.356702,15.798325,0.100001,0.100001,5.892661,0.100004,0.100013,0.100008,0.100009,0.100007,6.26946,0.100000,0.100007,7.518200
Topic1,4.787660,0.100006,6.099944,0.100002,17.392305,0.100000,2.549026,0.100000,0.100000,1.389002,0.100009,12.096774,0.100000,0.100011,0.100002,6.569095,0.100009,0.100002,10.837115,0.100011,0.100000,0.100013,0.100003,0.100009,0.100005,7.099937,0.100011,0.100004,9.078949,0.100001,0.100000,5.099941,2.313473,0.100014,1.118490,0.100019,0.665867,0.100008,0.100002,0.100006,...,0.100003,6.396251,0.100009,0.100000,12.593477,0.100004,19.651406,0.100002,4.204198,0.100000,0.100003,0.100001,0.100011,0.100004,6.438339,10.235655,0.100041,0.100005,0.100001,0.100000,0.100005,0.100012,0.448008,0.100004,0.100013,14.262987,0.100005,0.100019,1.099884,1.131799,9.307314,0.100000,96.628656,104.243969,0.100000,8.100049,0.10000,0.100038,7.036618,6.235242
Topic2,0.100042,38.099955,0.100041,0.100005,1.012246,2.781585,3.790090,0.100001,0.100002,4.992357,0.100009,0.100000,0.100009,0.100015,0.100003,0.100006,5.099935,0.100001,0.100009,0.100009,0.100002,9.395922,0.100005,2.730949,0.100005,0.100000,0.100009,0.100007,11.336283,8.099999,0.100004,0.100003,0.100000,8.464212,0.117551,0.100014,0.100006,35.099934,0.100000,0.100004,...,11.100047,0.100020,0.298022,0.100003,54.150411,0.100015,0.759670,0.100000,6.540445,7.146674,9.945131,0.100001,0.100018,0.100007,0.100002,0.100005,2.193090,0.100003,7.808035,0.100002,0.100006,5.100001,0.100006,2.146602,0.100004,0.100004,0.100006,0.100004,0.100003,0.100003,0.100001,0.100005,151.647519,9.850685,15.099946,0.100000,4.33376,0.100001,5.150834,0.100002
Topic3,0.100000,0.100004,0.100002,0.100004,0.100002,3.099966,1.491475,0.100008,2.402445,0.100003,4.744531,0.100007,2.099955,0.100010,0.100002,0.100004,0.100007,0.107594,2.190402,10.099967,0.100002,0.100027,0.100001,12.580662,0.100013,0.100000,4.789617,97.628419,4.199601,0.100005,0.100039,0.100000,0.100000,7.301850,0.100003,40.502643,13.964181,0.100010,0.100000,0.100015,...,0.100001,0.100003,0.100000,0.325284,3.522495,0.100008,0.100007,0.100024,0.100000,0.100009,0.100008,8.512994,2.760234,0.100003,0.100006,4.373522,0.100002,0.100000,0.100008,0.100020,0.100001,0.100000,0.100025,0.100007,0.100006,0.100000,0.100011,0.100007,0.100004,0.100004,0.100002,0.100001,3.963511,0.100004,0.100023,0.100006,0.10000,0.100002,0.100003,0.100006
Topic4,2.099836,0.100003,0.100024,0.100001,0.100005,0.100008,0.100013,0.100003,0.100005,0.100006,0.100007,0.100019,0.100000,18.580605,0.100003,9.975956,0.100000,16.092380,0.100003,0.100002,16.099962,3.436543,0.100004,3.829261,0.100004,0.100028,0.100022,16.465893,9.408118,0.100156,17.632401,0.100007,0.100004,4.810899,0.100000,0.100003,0.788608,0.100010,0.100000,0.100000,...,0.100011,5.236671,0.100005,0.100000,0.100006,3.473962,103.162236,0.100001,3.076738,0.100008

In [30]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,need,employee,care,process,really,make,people,listen,interview,think,transparency,role,long,level,skill
Topic 1,worker,leadership,work,need,customer,improve,team,quality,talk,know,friendly,focus,management,employee,supportive
Topic 2,people,work,new,thing,challenge,good,client,learn,growth,able,environment,company,career,positive,provide
Topic 3,benefit,good,insurance,health,fair,compensation,salary,business,package,opportunity,great,plan,employee,fun,interviewer
Topic 4,work,pay,time,make,hard,feel,hour,offer,low,employee,money,company,day,leave,market
Topic 5,employee,great,manager,company,leadership,value,treat,respect,change,raise,member,stock,increase,strong,goal
Topic 6,management,communication,coworker,easy,nice,work,good,interview,people,employee,project,professional,policy,place,fast
Topic 7,team,work,good,family,support,great,stop,like,culture,new,job,break,try,improve,trust
Topic 8,job,just,hire,people,work,day,life,happy,ask,make,say,experience,use,balance,interview
Topic 9,good,company,help,culture,love,environment,people,work,open,diversity,group,different,feel,way,food


# Predic Topics on New Reviews

In [31]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["gender equality"]
topic, prob_scores = predict_topic(text = mytext)
print(topic)

['need', 'employee', 'care', 'process', 'really', 'make', 'people', 'listen', 'interview', 'think', 'transparency', 'role', 'long', 'level', 'skill']
